In [63]:
# To import functions from utils.py, which is in the parent diretory

import sys
import os

current_dir = os.path.dirname(os.path.abspath('__file__')) 

parent_dir = os.path.dirname(current_dir)

sys.path.append(parent_dir)

In [64]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from utils import delete_patients_with_the_same_GUID, delete_patients_without_GUID, averaging_scores
import warnings
warnings.filterwarnings("ignore")

sns.set_style("whitegrid")

# Appending proteomic data

In [65]:
data=pd.read_csv("cleaned_data.csv")

In [66]:
data[data.participant_id=="BF-1045"]

,participant_id,visit_name,visit_month,mds_updrs_part_i_summary_score,mds_updrs_part_ii_summary_score,mds_updrs_part_iii_summary_score,caff_drinks_ever_used_regularly,caff_drinks_current_use,age_at_baseline,sex,...,roi6_right_caudal_Eigenvalue3,roi6_right_caudal_Fractional Anisotropy,ref1_left_reference_Eigenvalue1,ref1_left_reference_Eigenvalue2,ref1_left_reference_Eigenvalue3,ref1_left_reference_Fractional Anisotropy,ref2_right_reference_Eigenvalue1,ref2_right_reference_Eigenvalue2,ref2_right_reference_Eigenvalue3,ref2_right_reference_Fractional Anisotropy
1012,BF-1045,M0,0,7.0,12.0,39.0,NaN,NaN,72,Male,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Take a look at the `cfs_abeta_tau_ptau_data` before appending

In [67]:
cfs_abeta_tau_ptau_data=pd.read_csv("../new-data/releases_2023_v4release_1027_clinical_Biospecimen_analyses_CSF_abeta_tau_ptau.csv")
cfs_abeta_tau_ptau_cleaned_data=delete_patients_with_the_same_GUID(cfs_abeta_tau_ptau_data)
cfs_abeta_tau_ptau_cleaned_data=cfs_abeta_tau_ptau_cleaned_data.drop("GUID", axis=1)

In [68]:
cfs_abeta_tau_ptau_cleaned_data.head(5)

,participant_id,visit_name,visit_month,sample_type,test_name,test_value,test_units
0,BF-1001,M0_5,0.5,CSF,Abeta,361.06,pg/mL
1,BF-1001,M0_5,0.5,CSF,p-Tau,29.61,pg/mL
2,BF-1001,M0_5,0.5,CSF,Tau,34.73,pg/mL
3,BF-1002,M0_5,0.5,CSF,Abeta,230.66,pg/mL
4,BF-1002,M0_5,0.5,CSF,p-Tau,4.30,pg/mL


Pivot the table

In [69]:
cfs_abeta_tau_ptau_cleaned_and_pivotted_data=cfs_abeta_tau_ptau_cleaned_data.pivot(columns=["sample_type", "test_name", "test_units"], index=["participant_id", "visit_name","visit_month"])
cfs_abeta_tau_ptau_cleaned_and_pivotted_data.columns = ['_'.join(map(str, col)).strip() for col in cfs_abeta_tau_ptau_cleaned_and_pivotted_data.columns.values]
cfs_abeta_tau_ptau_cleaned_and_pivotted_data = cfs_abeta_tau_ptau_cleaned_and_pivotted_data.reset_index()
cfs_abeta_tau_ptau_cleaned_and_pivotted_data.head(5)

,participant_id,visit_name,visit_month,test_value_CSF_Abeta_pg/mL,test_value_CSF_p-Tau_pg/mL,test_value_CSF_Tau_pg/mL
0,BF-1001,M0_5,0.5,361.06,29.61,34.73
1,BF-1002,M0_5,0.5,230.66,4.30,NaN
2,BF-1003,M0_5,0.5,257.50,6.26,20.35
3,BF-1004,M0_5,0.5,449.67,36.84,49.26
4,BF-1005,M0_5,0.5,418.91,13.20,38.65


In [70]:
display(cfs_abeta_tau_ptau_cleaned_and_pivotted_data.info())


print("----------------------------------")
print("Number of times every participant attended the survey")
display(cfs_abeta_tau_ptau_cleaned_and_pivotted_data.participant_id.value_counts().value_counts())

print("----------------------------------")
print("Detailed time of the survey:")
display(cfs_abeta_tau_ptau_cleaned_and_pivotted_data.visit_month.value_counts())

print("----------------------------------")
print("Any repeated test in a single month or during screening? (#2, #3, etc. indicates repeated visits.)")
display(cfs_abeta_tau_ptau_cleaned_and_pivotted_data.visit_name.value_counts())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3262 entries, 0 to 3261
Data columns (total 6 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   participant_id              3262 non-null   object 
 1   visit_name                  3262 non-null   object 
 2   visit_month                 3259 non-null   float64
 3   test_value_CSF_Abeta_pg/mL  3229 non-null   float64
 4   test_value_CSF_p-Tau_pg/mL  3004 non-null   float64
 5   test_value_CSF_Tau_pg/mL    3203 non-null   float64
dtypes: float64(4), object(2)
memory usage: 153.0+ KB


None

----------------------------------
Number of times every participant attended the survey


count
1     388
5     260
4     188
2     137
3     118
8      10
10      9
6       4
Name: count, dtype: int64

----------------------------------
Detailed time of the survey:


visit_month
0.0     926
12.0    656
6.0     611
24.0    497
36.0    358
0.5     191
3.0       8
18.0      7
30.0      4
9.0       1
Name: count, dtype: int64

----------------------------------
Any repeated test in a single month or during screening? (#2, #3, etc. indicates repeated visits.)


visit_name
M0       907
M12      637
M6       593
M24      482
M36      344
M0_5     191
M0#2      19
M12#2     19
M6#2      18
M24#2     15
M36#2     14
M3         8
M18        7
M30        4
LOG        3
M9         1
Name: count, dtype: int64

Clearly this is a time dependent data, so we drop all the results with `visit_name=LOG`

Now we take the average of test results for tests conducted in the same month

In [71]:
cfs_abeta_tau_ptau_cleaned_and_pivotted_data=averaging_scores(cfs_abeta_tau_ptau_cleaned_and_pivotted_data, ["test_value_CSF_Abeta_pg/mL","test_value_CSF_p-Tau_pg/mL","test_value_CSF_Tau_pg/mL"])

Processing regular visit records by participant and month...
Consolidated 3262 regular records into 3174 records.


In [72]:
data.visit_name.value_counts()

visit_name
M0      3983
M12     2098
M6      1976
M24     1951
M18     1718
M36     1558
M30     1451
SC       996
M48      850
M42      792
M60      628
M3       517
M54      472
M9       461
M84      342
M72      306
M96      185
M108      30
Name: count, dtype: int64

We drop the `visit_name` from `cfs_abeta_tau_ptau_cleaned_and_pivotted_data` and merge it to `data`.

In [73]:
cfs_abeta_tau_ptau_cleaned_and_pivotted_data = cfs_abeta_tau_ptau_cleaned_and_pivotted_data.drop("visit_name", axis=1)
data = data.merge(
    cfs_abeta_tau_ptau_cleaned_and_pivotted_data,
    on=["participant_id", "visit_month"],
    how="left"
)

In [74]:
data.head(5)

,participant_id,visit_name,visit_month,mds_updrs_part_i_summary_score,mds_updrs_part_ii_summary_score,mds_updrs_part_iii_summary_score,caff_drinks_ever_used_regularly,caff_drinks_current_use,age_at_baseline,sex,...,ref1_left_reference_Eigenvalue2,ref1_left_reference_Eigenvalue3,ref1_left_reference_Fractional Anisotropy,ref2_right_reference_Eigenvalue1,ref2_right_reference_Eigenvalue2,ref2_right_reference_Eigenvalue3,ref2_right_reference_Fractional Anisotropy,test_value_CSF_Abeta_pg/mL,test_value_CSF_p-Tau_pg/mL,test_value_CSF_Tau_pg/mL
0,LC-460010,SC,-1,1.0,0.0,3.0,NaN,NaN,78,Female,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,PP-10874,SC,-1,0.0,0.0,2.0,NaN,NaN,73,Female,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,PP-12499,SC,-1,4.0,2.0,0.0,NaN,NaN,64,Male,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,PP-12593,SC,-1,4.0,3.0,2.0,Yes,Yes,66,Female,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,PP-13039,SC,-1,5.0,3.0,0.0,Yes,Yes,67,Male,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [75]:
data[data.participant_id=="BF-1045"]

,participant_id,visit_name,visit_month,mds_updrs_part_i_summary_score,mds_updrs_part_ii_summary_score,mds_updrs_part_iii_summary_score,caff_drinks_ever_used_regularly,caff_drinks_current_use,age_at_baseline,sex,...,ref1_left_reference_Eigenvalue2,ref1_left_reference_Eigenvalue3,ref1_left_reference_Fractional Anisotropy,ref2_right_reference_Eigenvalue1,ref2_right_reference_Eigenvalue2,ref2_right_reference_Eigenvalue3,ref2_right_reference_Fractional Anisotropy,test_value_CSF_Abeta_pg/mL,test_value_CSF_p-Tau_pg/mL,test_value_CSF_Tau_pg/mL
1012,BF-1045,M0,0,7.0,12.0,39.0,NaN,NaN,72,Male,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


We carry out similar process for other proteomic data.

In [76]:
csf_beta_glucocerebrosidase_data=pd.read_csv("../new-data/releases_2023_v4release_1027_clinical_Biospecimen_analyses_CSF_beta_glucocerebrosidase.csv")
csf_beta_glucocerebrosidase_cleaned_data=delete_patients_with_the_same_GUID(csf_beta_glucocerebrosidase_data)
csf_beta_glucocerebrosidase_cleaned_data=csf_beta_glucocerebrosidase_cleaned_data.drop("GUID", axis=1)

In [77]:
csf_beta_glucocerebrosidase_cleaned_data.head(5)

,participant_id,visit_name,visit_month,sample_type,test_name,test_value,test_units
0,BF-1005,M0_5,0.5,CSF,beta-glucocerebrosidase,15.06,Average (pmol/min/mL)
1,BF-1005,M0_5,0.5,CSF,beta-glucocerebrosidase,1.74,%CV
2,BF-1008,M0_5,0.5,CSF,beta-glucocerebrosidase,4.31,%CV
3,BF-1008,M0_5,0.5,CSF,beta-glucocerebrosidase,9.97,Average (pmol/min/mL)
4,BF-1009,M0_5,0.5,CSF,beta-glucocerebrosidase,12.13,Average (pmol/min/mL)


Pivot the table

In [78]:
csf_beta_glucocerebrosidase_cleaned_and_pivotted_data=csf_beta_glucocerebrosidase_cleaned_data.pivot(columns=["sample_type", "test_name", "test_units"], index=["participant_id", "visit_name","visit_month"])
csf_beta_glucocerebrosidase_cleaned_and_pivotted_data.columns = ['_'.join(map(str, col)).strip() for col in csf_beta_glucocerebrosidase_cleaned_and_pivotted_data.columns.values]
csf_beta_glucocerebrosidase_cleaned_and_pivotted_data = csf_beta_glucocerebrosidase_cleaned_and_pivotted_data.reset_index()
csf_beta_glucocerebrosidase_cleaned_and_pivotted_data.head(5)

,participant_id,visit_name,visit_month,test_value_CSF_beta-glucocerebrosidase_Average (pmol/min/mL),test_value_CSF_beta-glucocerebrosidase_%CV
0,BF-1005,M0_5,0.5,15.06,1.74
1,BF-1008,M0_5,0.5,9.97,4.31
2,BF-1009,M0_5,0.5,12.13,5.20
3,BF-1010,M0_5,0.5,10.68,7.25
4,BF-1011,M0_5,0.5,9.30,2.67


In [79]:
display(csf_beta_glucocerebrosidase_cleaned_and_pivotted_data.info())


print("----------------------------------")
print("Number of times every participant attended the survey")
display(csf_beta_glucocerebrosidase_cleaned_and_pivotted_data.participant_id.value_counts().value_counts())

print("----------------------------------")
print("Detailed time of the survey:")
display(csf_beta_glucocerebrosidase_cleaned_and_pivotted_data.visit_month.value_counts())

print("----------------------------------")
print("Any repeated test in a single month or during screening? (#2, #3, etc. indicates repeated visits.)")
display(csf_beta_glucocerebrosidase_cleaned_and_pivotted_data.visit_name.value_counts())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 139 entries, 0 to 138
Data columns (total 5 columns):
 #   Column                                                        Non-Null Count  Dtype  
---  ------                                                        --------------  -----  
 0   participant_id                                                139 non-null    object 
 1   visit_name                                                    139 non-null    object 
 2   visit_month                                                   137 non-null    float64
 3   test_value_CSF_beta-glucocerebrosidase_Average (pmol/min/mL)  139 non-null    float64
 4   test_value_CSF_beta-glucocerebrosidase_%CV                    139 non-null    float64
dtypes: float64(3), object(2)
memory usage: 5.6+ KB


None

----------------------------------
Number of times every participant attended the survey


count
1    139
Name: count, dtype: int64

----------------------------------
Detailed time of the survey:


visit_month
0.5    137
Name: count, dtype: int64

----------------------------------
Any repeated test in a single month or during screening? (#2, #3, etc. indicates repeated visits.)


visit_name
M0_5    137
LOG       2
Name: count, dtype: int64

However, we observe here that the `csf_beta_glucocerebrosidase` tests are only conducted without visit or at month 0.5, and since month 0.5 does not appear as the `visit_month` of other features and targets, we discard the `csf_beta_glucocerebrosidase_data`.

In [80]:
other_analysis_data=pd.read_csv("../new-data/releases_2023_v4release_1027_clinical_Biospecimen_analyses_other.csv")
other_analysis_cleaned_data=delete_patients_with_the_same_GUID(other_analysis_data)
other_analysis_cleaned_data=other_analysis_cleaned_data.drop("GUID", axis=1)

In [81]:
other_analysis_cleaned_data.head(5)

,participant_id,visit_name,visit_month,sample_type,test_name,test_value,test_units
0,HB-PD_INVAA223GY7,M12,12,Plasma,Total Ceramide,5.08,ug/mL
1,HB-PD_INVAA223GY7,M12,12,Plasma,Total Glucosylceramide,4.69,ug/mL
2,HB-PD_INVAA223GY7,M12,12,Plasma,Total Lactosylceramide,3.63,ug/mL
3,HB-PD_INVAA223GY7,M12,12,Plasma,Total Sphingomyelin,189.98,ug/mL
4,HB-PD_INVAA769TBF,M0,0,Plasma,Total Ceramide,6.18,ug/mL


Pivot the table

In [82]:
other_analysis_cleaned_and_pivotted_data=other_analysis_cleaned_data.pivot(columns=["sample_type", "test_name", "test_units"], index=["participant_id", "visit_name","visit_month"])
other_analysis_cleaned_and_pivotted_data.columns = ['_'.join(map(str, col)).strip() for col in other_analysis_cleaned_and_pivotted_data.columns.values]
other_analysis_cleaned_and_pivotted_data = other_analysis_cleaned_and_pivotted_data.reset_index()
other_analysis_cleaned_and_pivotted_data.head(5)

,participant_id,visit_name,visit_month,test_value_Plasma_Total Ceramide_ug/mL,test_value_Plasma_Total Glucosylceramide_ug/mL,test_value_Plasma_Total Lactosylceramide_ug/mL,test_value_Plasma_Total Sphingomyelin_ug/mL,test_value_CSF_H-Ferritin_ng/mL,test_value_Serum_H-Ferritin_ng/mL,test_value_RNA_Actin Beta_Stdev,...,"test_value_RNA_Protein Tyrosine Phosphatase, Non-Receptor Type 1_Stdev","test_value_RNA_Protein Tyrosine Phosphatase, Non-Receptor Type 1_Ct Avg","test_value_RNA_Protein Tyrosine Phosphatase, Receptor Type C_Stdev","test_value_RNA_Protein Tyrosine Phosphatase, Receptor Type C_Ct Avg",test_value_RNA_Urea transporter 1_Stdev,test_value_RNA_Urea transporter 1_Ct Avg,test_value_RNA_Zinc Finger Protein 134_Stdev,test_value_RNA_Zinc Finger Protein 134_Ct Avg,test_value_Plasma_H-Ferritin_ng/mL,test_value_CSF_a-syn_ng/mL
0,HB-PD_INVAA223GY7,M12,12,5.08,4.69,3.63,189.98,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,HB-PD_INVAA769TBF,M0,0,6.18,4.07,5.80,295.57,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,HB-PD_INVAB465GYE,M0,0,6.00,4.39,4.98,254.48,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,HB-PD_INVAD802MY3,M0,0,5.44,7.16,7.74,222.37,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,HB-PD_INVAE296YP8,M0,0,4.55,3.00,2.53,169.40,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [83]:
display(other_analysis_cleaned_and_pivotted_data.info())


print("----------------------------------")
print("Number of times every participant attended the survey")
display(other_analysis_cleaned_and_pivotted_data.participant_id.value_counts().value_counts())

print("----------------------------------")
print("Detailed time of the survey:")
display(other_analysis_cleaned_and_pivotted_data.visit_month.value_counts())

print("----------------------------------")
print("Any repeated test in a single month or during screening? (#2, #3, etc. indicates repeated visits.)")
display(other_analysis_cleaned_and_pivotted_data.visit_name.value_counts())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 926 entries, 0 to 925
Data columns (total 41 columns):
 #   Column                                                                   Non-Null Count  Dtype  
---  ------                                                                   --------------  -----  
 0   participant_id                                                           926 non-null    object 
 1   visit_name                                                               926 non-null    object 
 2   visit_month                                                              926 non-null    int64  
 3   test_value_Plasma_Total Ceramide_ug/mL                                   322 non-null    float64
 4   test_value_Plasma_Total Glucosylceramide_ug/mL                           581 non-null    float64
 5   test_value_Plasma_Total Lactosylceramide_ug/mL                           322 non-null    float64
 6   test_value_Plasma_Total Sphingomyelin_ug/mL                              3

None

----------------------------------
Number of times every participant attended the survey


count
1    407
2    162
3     61
4      3
Name: count, dtype: int64

----------------------------------
Detailed time of the survey:


visit_month
0      695
12     118
24      93
60      10
72       3
84       2
108      2
96       2
48       1
Name: count, dtype: int64

----------------------------------
Any repeated test in a single month or during screening? (#2, #3, etc. indicates repeated visits.)


visit_name
M0      563
M0#2    132
M12     118
M24      93
M60      10
M72       3
M84       2
M108      2
M96       2
M48       1
Name: count, dtype: int64

Take average for tests happened at the same month and merge the results to `data`.

In [84]:
cols_to_average=other_analysis_cleaned_and_pivotted_data.columns.tolist()
cols_to_average.remove("participant_id")
cols_to_average.remove("visit_name")
cols_to_average.remove("visit_month")
other_analysis_cleaned_and_pivotted_data=averaging_scores(other_analysis_cleaned_and_pivotted_data, cols_to_average)

Processing regular visit records by participant and month...
Consolidated 926 regular records into 794 records.


In [85]:
data[data.participant_id=="BF-1045"]

,participant_id,visit_name,visit_month,mds_updrs_part_i_summary_score,mds_updrs_part_ii_summary_score,mds_updrs_part_iii_summary_score,caff_drinks_ever_used_regularly,caff_drinks_current_use,age_at_baseline,sex,...,ref1_left_reference_Eigenvalue2,ref1_left_reference_Eigenvalue3,ref1_left_reference_Fractional Anisotropy,ref2_right_reference_Eigenvalue1,ref2_right_reference_Eigenvalue2,ref2_right_reference_Eigenvalue3,ref2_right_reference_Fractional Anisotropy,test_value_CSF_Abeta_pg/mL,test_value_CSF_p-Tau_pg/mL,test_value_CSF_Tau_pg/mL
1012,BF-1045,M0,0,7.0,12.0,39.0,NaN,NaN,72,Male,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [86]:
other_analysis_cleaned_and_pivotted_data = other_analysis_cleaned_and_pivotted_data.drop("visit_name", axis=1)
data = data.merge(
    other_analysis_cleaned_and_pivotted_data,
    on=["participant_id", "visit_month"],
    how="left"
)

In [87]:
data[data.participant_id=="BF-1045"]

,participant_id,visit_name,visit_month,mds_updrs_part_i_summary_score,mds_updrs_part_ii_summary_score,mds_updrs_part_iii_summary_score,caff_drinks_ever_used_regularly,caff_drinks_current_use,age_at_baseline,sex,...,"test_value_RNA_Protein Tyrosine Phosphatase, Non-Receptor Type 1_Stdev","test_value_RNA_Protein Tyrosine Phosphatase, Non-Receptor Type 1_Ct Avg","test_value_RNA_Protein Tyrosine Phosphatase, Receptor Type C_Stdev","test_value_RNA_Protein Tyrosine Phosphatase, Receptor Type C_Ct Avg",test_value_RNA_Urea transporter 1_Stdev,test_value_RNA_Urea transporter 1_Ct Avg,test_value_RNA_Zinc Finger Protein 134_Stdev,test_value_RNA_Zinc Finger Protein 134_Ct Avg,test_value_Plasma_H-Ferritin_ng/mL,test_value_CSF_a-syn_ng/mL
1012,BF-1045,M0,0,7.0,12.0,39.0,NaN,NaN,72,Male,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [88]:
plasma_analysis_data=pd.read_csv("../new-data/releases_2023_v4release_1027_clinical_Biospecimen_analyses_SomaLogic_plasma.csv")
plasma_analysis_cleaned_data=delete_patients_with_the_same_GUID(plasma_analysis_data)
plasma_analysis_cleaned_data=plasma_analysis_cleaned_data.drop("GUID", axis=1)
plasma_analysis_cleaned_data.head(5)

,participant_id,visit_name,visit_month,sample_type,test_name,test_value,test_units
0,BF-1045,M0,0.0,Plasma,14-3-3 protein beta/alpha,20989.2,RFU
1,BF-1045,M0,0.0,Plasma,14-3-3 protein epsilon,788.9,RFU
2,BF-1045,M0,0.0,Plasma,14-3-3 protein family,6006.0,RFU
3,BF-1045,M0,0.0,Plasma,14-3-3 protein sigma,1066.5,RFU
4,BF-1045,M0,0.0,Plasma,14-3-3 protein theta,1249.0,RFU


In [89]:
plasma_analysis_cleaned_and_pivotted_data=plasma_analysis_cleaned_data.pivot(columns=["sample_type", "test_name", "test_units"], index=["participant_id", "visit_name","visit_month"])
plasma_analysis_cleaned_and_pivotted_data.columns = ['_'.join(map(str, col)).strip() for col in plasma_analysis_cleaned_and_pivotted_data.columns.values]
plasma_analysis_cleaned_and_pivotted_data = plasma_analysis_cleaned_and_pivotted_data.reset_index()
plasma_analysis_cleaned_and_pivotted_data.head(5)

,participant_id,visit_name,visit_month,test_value_Plasma_14-3-3 protein beta/alpha_RFU,test_value_Plasma_14-3-3 protein epsilon_RFU,test_value_Plasma_14-3-3 protein family_RFU,test_value_Plasma_14-3-3 protein sigma_RFU,test_value_Plasma_14-3-3 protein theta_RFU,test_value_Plasma_14-3-3 protein zeta/delta_RFU,test_value_Plasma_15-hydroxyprostaglandin dehydrogenase [NAD(+)]_RFU,...,test_value_Plasma_von Willebrand factor_RFU,test_value_Plasma_VPS10 domain-containing receptor SorCS2_RFU,"test_value_Plasma_WAP, kazal, immunoglobulin, kunitz and NTR domain-containing protein 1_RFU","test_value_Plasma_WAP, Kazal, immunoglobulin, Kunitz and NTR domain-containing protein 2_RFU",test_value_Plasma_WNT1-inducible-signaling pathway protein 1_RFU,test_value_Plasma_WNT1-inducible-signaling pathway protein 3_RFU,test_value_Plasma_Wnt inhibitory factor 1_RFU,test_value_Plasma_Xaa-Pro aminopeptidase 1_RFU,test_value_Plasma_X-linked interleukin-1 receptor accessory protein-like 2_RFU,test_value_Plasma_X-ray repair cross-complementing protein 6_RFU
0,BF-1045,M0,0.0,20989.2,788.9,6006.0,1066.5,1249.0,157378.6,1412.4,...,5782.4,403.0,1360.5,7366.4,6339.3,1010.3,3040.5,4272.6,822.3,215.7
1,BF-1045,M0_5,0.5,18988.3,815.1,3906.7,855.7,1335.1,179783.2,1381.2,...,4702.0,396.4,1387.6,7102.7,4968.9,1109.2,2916.6,3684.9,865.8,212.7
2,BF-1051,M0,0.0,123631.6,806.4,32947.8,1505.7,3675.3,292247.5,3126.4,...,9526.4,325.5,565.4,2613.4,8214.4,1242.4,1903.5,18962.3,955.4,722.3
3,BF-1051,M0_5,0.5,160348.4,1035.0,34973.8,1348.5,3518.7,328131.6,3443.2,...,6518.6,316.1,537.6,2598.3,7749.4,1319.2,2127.0,31959.2,961.5,923.0
4,BF-1052,M0,0.0,139951.1,830.7,33294.9,1264.6,3345.7,323516.7,3081.0,...,12725.0,585.7,552.7,3095.8,6284.4,1214.2,2046.5,17531.4,956.7,337.9


In [90]:
display(plasma_analysis_cleaned_and_pivotted_data.info())


print("----------------------------------")
print("Number of times every participant attended the survey")
display(plasma_analysis_cleaned_and_pivotted_data.participant_id.value_counts().value_counts())

print("----------------------------------")
print("Detailed time of the survey:")
display(plasma_analysis_cleaned_and_pivotted_data.visit_month.value_counts())

print("----------------------------------")
print("Any repeated test in a single month or during screening? (#2, #3, etc. indicates repeated visits.)")
display(plasma_analysis_cleaned_and_pivotted_data.visit_name.value_counts())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Columns: 1308 entries, participant_id to test_value_Plasma_X-ray repair cross-complementing protein 6_RFU
dtypes: float64(1306), object(2)
memory usage: 204.5+ KB


None

----------------------------------
Number of times every participant attended the survey


count
2    10
Name: count, dtype: int64

----------------------------------
Detailed time of the survey:


visit_month
0.0    10
0.5    10
Name: count, dtype: int64

----------------------------------
Any repeated test in a single month or during screening? (#2, #3, etc. indicates repeated visits.)


visit_name
M0      10
M0_5    10
Name: count, dtype: int64

There is no repeated test, and we will only keep the result from the test at month 0.

In [91]:
plasma_analysis_cleaned_and_pivotted_data = plasma_analysis_cleaned_and_pivotted_data.drop("visit_name", axis=1)
data = data.merge(
    plasma_analysis_cleaned_and_pivotted_data,
    on=["participant_id", "visit_month"],
    how="left"
)

In [92]:
data[data.participant_id=="BF-1045"]

,participant_id,visit_name,visit_month,mds_updrs_part_i_summary_score,mds_updrs_part_ii_summary_score,mds_updrs_part_iii_summary_score,caff_drinks_ever_used_regularly,caff_drinks_current_use,age_at_baseline,sex,...,test_value_Plasma_von Willebrand factor_RFU,test_value_Plasma_VPS10 domain-containing receptor SorCS2_RFU,"test_value_Plasma_WAP, kazal, immunoglobulin, kunitz and NTR domain-containing protein 1_RFU","test_value_Plasma_WAP, Kazal, immunoglobulin, Kunitz and NTR domain-containing protein 2_RFU",test_value_Plasma_WNT1-inducible-signaling pathway protein 1_RFU,test_value_Plasma_WNT1-inducible-signaling pathway protein 3_RFU,test_value_Plasma_Wnt inhibitory factor 1_RFU,test_value_Plasma_Xaa-Pro aminopeptidase 1_RFU,test_value_Plasma_X-linked interleukin-1 receptor accessory protein-like 2_RFU,test_value_Plasma_X-ray repair cross-complementing protein 6_RFU
1012,BF-1045,M0,0,7.0,12.0,39.0,NaN,NaN,72,Male,...,5782.4,403.0,1360.5,7366.4,6339.3,1010.3,3040.5,4272.6,822.3,215.7


In [93]:
data.to_csv("complete_data.csv", index=False)